In [6]:
import numpy as np
import pandas as pd
import yfinance as yf

from backtesting import Backtest, Strategy 
from backtesting.lib import crossover
from backtesting.test import SMA

In [7]:
msft = yf.Ticker("msft")

In [18]:
hist = msft.history(period="2y")

In [19]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-06-17,129.761365,130.837565,129.663522,129.976608,14517800,0.0,0
2019-06-18,131.287612,132.314905,130.681027,132.236633,25934500,0.0,0
2019-06-19,132.080109,132.989988,130.915845,132.755188,23744400,0.0,0
2019-06-20,134.477070,134.682534,132.784493,133.987885,33042600,0.0,0
2019-06-21,133.625913,134.751033,133.508513,134.007477,36727900,0.0,0
...,...,...,...,...,...,...,...
2021-06-10,254.289993,257.459991,253.669998,257.239990,24563600,0.0,0
2021-06-11,257.989990,258.489990,256.609985,257.890015,18995100,0.0,0
2021-06-14,257.899994,259.950012,256.799988,259.890015,19150500,0.0,0


In [20]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 30

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

In [22]:
bt = Backtest(
    hist,
    SmaCross,
    cash=1000000,
    commission=0.005,
    margin=1.0,
    trade_on_close=True,
    exclusive_orders=True
)

output = bt.run()
print(output)

Start                     2019-06-17 00:00:00
End                       2021-06-16 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   69.306931
Equity Final [$]               1359413.856302
Equity Peak [$]                1694419.114608
Return [%]                          35.941386
Buy & Hold Return [%]               99.728086
Return (Ann.) [%]                   16.558464
Volatility (Ann.) [%]               25.723429
Sharpe Ratio                         0.643711
Sortino Ratio                        1.079951
Calmar Ratio                         0.687981
Max. Drawdown [%]                  -24.068184
Avg. Drawdown [%]                   -3.930295
Max. Drawdown Duration      287 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   11
Win Rate [%]                        45.454545
Best Trade [%]                       27.72858
Worst Trade [%]                     -9.005099
Avg. Trade [%]                    

In [24]:
bt.plot()
